In [15]:
import numpy as np
import pandas as pd
import requests

from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans

In [ ]:
# Import Data
print('Load Description Embeddings')
desc_emb = np.load('../data/desc_emb.npz')
traindata_desc_embeddings = dict(zip(("desc_embeddings"), (desc_emb[k] for k in desc_emb)))['d']
print(traindata_desc_embeddings.shape)

print('Load Description Labels, Address Labels and Name Labels')
NFT_attributes = np.load('../data/NFT_attributes.npz')
d = dict(zip(("description_list","contract_addresses","name_list"), (NFT_attributes[k] for k in NFT_attributes)))
description_list = d['description_list']
contract_addresses = d['contract_addresses']
name_list = d['name_list']
print(description_list.shape)

train_desc_address = list(zip(contract_addresses, traindata_desc_embeddings, description_list))


In [ ]:
def find_nearest_neighbor(embedding, embedding_list):
    min_index = 0
    min_distance = 100
    for j, dj in enumerate(embedding_list):
        distance = np.sum(np.square((embedding - dj)))
        if distance < min_distance and distance > 1e-6:
            min_distance = distance
            min_index = j
    return min_index, embedding_list[min_index], min_distance


In [17]:
# Here we store the address of the person logging in with MetaMask 
OWNER = '0xe11BFCBDd43745d4Aa6f4f18E24aD24f4623af04'

URL = f'https://eth-mainnet.alchemyapi.io/v2/demo/getNFTs/?owner={OWNER}'
Owner_NFTs = requests.get(URL).json()['ownedNfts']

#Format is (Contractaddress, tokenId)
owner_nfts = []

for nft in Owner_NFTs:
    nft_contract_address = nft['contract']['address'] 
    token_id = nft['id']['tokenId']
    owner_nfts.append((nft_contract_address, token_id))

In [18]:
# Get Description of All NFTs of Owner
Owner_nft_fullAttributes = []
Owner_Descriptions = []

for nft in owner_nfts:
    nft_contract_address = nft[0]
    token_id = nft[1]
    URL = f"https://eth-mainnet.alchemyapi.io/v2/demo/getNFTMetadata?contractAddress={nft_contract_address}&tokenId={token_id}"
    NFT = requests.get(URL)
    try:
        NFTjson = NFT.json()
    except:
        continue
    if (type(NFTjson) is dict and "metadata" in NFTjson.keys() and type(NFTjson["metadata"]) is dict and "description" in NFTjson["metadata"].keys()):
        description = NFTjson["metadata"]["description"]
        Owner_Descriptions.append(description)
        Owner_nft_fullAttributes.append((nft_contract_address, token_id, description))

print(Owner_nft_fullAttributes[0])

('0x000386e3f7559d9b6a2f5c46b4ad1a9587d59dc3', '0x0000000000000000000000000000000000000000000000000000000000000003', 'COUNTDOWN OVER. MINTING LIVE.  [https://nikentfs.xyz](https://nikentfs.xyz)')


In [23]:
# Option 1: User then chooses a specific NFT and we provide X (e.g. X = 5) recommendations
X = 5

#Set dependent on User Input
owner_nfts_withDesc_index = 0
nft_description = Owner_nft_fullAttributes[owner_nfts_withDesc_index][2]

# Later on here we will just load out finetuned model from above
model = SentenceTransformer('bert-base-nli-mean-tokens')

#description embeddings
nft_description_embedding = model.encode(nft_description, normalize_embeddings=True, show_progress_bar=True, batch_size = 64)

print(f'shape of the embedding of the description: {nft_description_embedding.shape}')

#now a quadruple of distance, embedded desc, desc and address but later will also have id
recommendations = []
total_recommendations = 0

for i, j in enumerate(train_desc_address):
    train_address = j[0]
    train_description_embedding = j[1]
    train_description = j[2]

    distance = np.sum(np.square((train_description_embedding - nft_description_embedding)))

    if (i < X and distance > 1e-6):
        recommendations.append((distance, train_description_embedding, train_description, train_address))
        recommendations.sort(key = lambda x: x[0])
    else:
        highest_distance = recommendations[X-1][0]
        if (distance < highest_distance and distance > 1e-6):
            recommendations[X-1] = (distance, train_description_embedding, train_description, train_address)
            recommendations.sort(key = lambda x: x[0])


print("Input NFT Description: ", nft_description)
print(f"Output NFT Descriptions for top {X} matches", list(zip(*recommendations))[2])

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.70it/s]


shape of the embedding of the description: (768,)
Input NFT Description:  COUNTDOWN OVER. MINTING LIVE.  [https://nikentfs.xyz](https://nikentfs.xyz)
Output NFT Descriptions for top 5 matches ('COUNTDOWN OVER. MINTING LIVE.  [https://ikusanft.xyz](https://ikusanft.xyz)', 'COUNTDOWN OVER. MINTING LIVE.  [https://apiensnft.xyz](https://apiensnft.xyz)', 'COUNTDOWN OVER. MINTING LIVE.  [https://haagendazsofficial.xyz](https://haagendazsofficial.xyz)', 'COUNTDOWN OVER. MINTING LIVE.  [https://haagendazsofficial.xyz](https://haagendazsofficial.xyz)', 'COUNTDOWN OVER. MINTING LIVE.  [https://haagendazsofficial.xyz](https://haagendazsofficial.xyz)')


In [ ]:
# Option 2: For all the NFTs the User have we find the X (e.g. X = 5) NFTs that are closest to any NFTs he already has

print('Loading Model')
model = SentenceTransformer('bert-base-nli-mean-tokens')

#description embeddings
print(f'Calculating embeddings for {len(Owner_Descriptions)} descriptions')
Owner_desc_embeddings = model.encode(Owner_Descriptions, normalize_embeddings=True, show_progress_bar=True, batch_size = 64)

number_of_OwnerNFTs = len(Owner_Descriptions)
number_of_recommendations = 5
recommendations, owner_desc = [], []

for i in range(number_of_recommendations):
    print(f'Calculating recommendations for {(i+1) % number_of_recommendations}/{number_of_recommendations}')
    Owner_Nft = Owner_nft_fullAttributes[i % number_of_recommendations]
    min_distance = 1e6
    recommendation = (1e6, 0, 0, 0)
    for j, train_desc_emb in enumerate(train_description_embedding):
        distance = np.sum(np.square((train_desc_emb - Owner_desc_embeddings[i])))
        if (distance < min_distance and distance > 1e-6):
            recommendation = (distance, train_description_embedding, train_description, train_address)
            min_distance = distance

    recommendations.append(recommendation)
    owner_desc.append(Owner_Nft[2])

for i in range(number_of_recommendations):
    print(f'\nRecommendation {i+1}/{number_of_recommendations}')
    print(f'Owner NFT: {owner_desc[i%number_of_recommendations]}')
    print(f"Recommended NFT Descriptions: ", list(zip(*recommendations))[2][i])


In [1]:
print('Calculating Recommendations for 1 NFT')

recommendatons = []

for i, owner_emb in enumerate(Owner_desc_embeddings):
    index, neighbor_emb, distance = find_nearest_neighbor(owner_emb, train_description_embedding)
    recommendatons.append(find_nearest_neighbor(distance, neighbor_emb, train_description, train_address))

    print(f'\n\nThe closest NFT for Owner_NFT_{i} is All_NFT_{index} with a distance of {min_distance}')
    print(f'\ndescription Owner_NFT_{i}: {Owner_Descriptions[:100]}')
    print(f'\ndescription All_NFT_{index}: {description_list[index][:100]}')

Calculating Recommendations for 1 NFT


NameError: name 'Owner_desc_embeddings' is not defined